Using EDA, help Stack Overflow implement the following features in its web application: 

When a user raises a question with a data science tag ('python', 'r', 'matlab', 'sas', 'excel', 'sql'), then Stack Overflow sends the notification to the relevant users.

When a user provides a data science tag ('python', 'r', 'matlab', 'sas', 'excel', 'sql') to their question, then Stack Overflow should come up with the expected approximate time to get it answered.

While a user raises a question, Stack Overflow wants to provide some suggestions to the users to get their queries answered quickly.

In [159]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
questions = pd.read_csv("https://stack-overflow-casestudy-datasets.s3.amazonaws.com/Questions.csv",encoding="ISO-8859-1") 
answers = pd.read_csv("https://stack-overflow-casestudy-datasets.s3.amazonaws.com/Answers.csv",encoding="ISO-8859-1")
tags = pd.read_csv("https://stack-overflow-casestudy-datasets.s3.amazonaws.com/Tags.csv",encoding="ISO-8859-1",dtype={'Tag': str})
users = pd.read_csv("https://stack-overflow-casestudy-datasets.s3.amazonaws.com/Users.csv",encoding="ISO-8859-1")

In [3]:
questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [4]:
tags.head()


,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [5]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750994 entries, 0 to 3750993
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 57.2+ MB


In [6]:
tags = tags.loc[(tags["Tag"] == "python") | (tags["Tag"] == "r")| (tags["Tag"] == "matlab") | (tags["Tag"] == "sql") | (tags["Tag"] == "sas") | (tags["Tag"] == "excel")]

In [7]:
tags['Tag'] = tags['Tag'].astype(str)


In [8]:
grouped_tags = tags.groupby("Id")['Tag'].apply(lambda tags: ' '.join(tags))


In [9]:
grouped_tags.head()

Id
120     sql
1970    sql
2120    sql
2840    sql
4110    sql
Name: Tag, dtype: object

In [10]:
grouped_tags.reset_index()

,Id,Tag
0,120,sql
1,1970,sql
2,2120,sql
3,2840,sql
4,4110,sql
...,...,...
132070,40142550,sql
132071,40142600,python
132072,40142840,python
132073,40142860,excel


In [11]:
grouped_tags_final = pd.DataFrame({'Id':grouped_tags.index, 'Tags':grouped_tags.values})

In [12]:
grouped_tags_final.head(5)

,Id,Tags
0,120,sql
1,1970,sql
2,2120,sql
3,2840,sql
4,4110,sql


In [13]:
df = questions.merge(grouped_tags_final, on='Id')

In [14]:
df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Tags
0,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql
1,1970,116.0,2008-08-05T06:39:31Z,NaN,10,What language do you use for PostgreSQL trigge...,<p>PostgreSQL is interesting in that it suppor...,sql
2,2120,383.0,2008-08-05T11:49:11Z,NaN,77,Convert HashBytes to VarChar,<p>I want to get the MD5 Hash of a string valu...,sql
3,2840,383.0,2008-08-05T20:57:00Z,NaN,32,Paging SQL Server 2005 Results,<p>How do I page results in SQL Server 2005?</...,sql
4,4110,369.0,2008-08-06T23:19:50Z,2014-12-30T11:43:01Z,7,What program can I use to generate diagrams of...,<p>I've been tasked with redesigning part of a...,sql


In [15]:
df.rename(columns={"Id":"Q_ID","OwnerUserId":"User_id_q","CreationDate":"Q_CreationDate","ClosedDate":"Q_ClosedDate","Score":"Q_Score",
"Title":"Q_Title","Body":"Questions"},inplace = True)

In [16]:
df.head()

,Q_ID,User_id_q,Q_CreationDate,Q_ClosedDate,Q_Score,Q_Title,Questions,Tags
0,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql
1,1970,116.0,2008-08-05T06:39:31Z,NaN,10,What language do you use for PostgreSQL trigge...,<p>PostgreSQL is interesting in that it suppor...,sql
2,2120,383.0,2008-08-05T11:49:11Z,NaN,77,Convert HashBytes to VarChar,<p>I want to get the MD5 Hash of a string valu...,sql
3,2840,383.0,2008-08-05T20:57:00Z,NaN,32,Paging SQL Server 2005 Results,<p>How do I page results in SQL Server 2005?</...,sql
4,4110,369.0,2008-08-06T23:19:50Z,2014-12-30T11:43:01Z,7,What program can I use to generate diagrams of...,<p>I've been tasked with redesigning part of a...,sql


In [17]:
answers.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


In [18]:
df1 = answers

In [19]:
df1.rename(columns={"Id":"A_ID","OwnerUserId":"User_id_a","CreationDate":"A_CreationDate","ParentId":"Q_ID","Score":"A_Score",
"Body":"Answers"},inplace = True)

In [20]:
df1.head()

,A_ID,User_id_a,A_CreationDate,Q_ID,A_Score,Answers
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


In [21]:
new = df.merge(df1,on="Q_ID")

In [22]:
new.head()

,Q_ID,User_id_q,Q_CreationDate,Q_ClosedDate,Q_Score,Q_Title,Questions,Tags,A_ID,User_id_a,A_CreationDate,A_Score,Answers
0,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql,124363,12734.0,2008-09-23T22:41:11Z,9,<p>The Jeff Prosise version from MSDN magazine...
1,1970,116.0,2008-08-05T06:39:31Z,NaN,10,What language do you use for PostgreSQL trigge...,<p>PostgreSQL is interesting in that it suppor...,sql,1971,269.0,2008-08-05T06:43:30Z,3,<p>When I wrote my first trigger it was in Per...
2,1970,116.0,2008-08-05T06:39:31Z,NaN,10,What language do you use for PostgreSQL trigge...,<p>PostgreSQL is interesting in that it suppor...,sql,3454,277.0,2008-08-06T14:26:08Z,5,"<p>I have only really used PL/pgSQL, but that ..."
3,1970,116.0,2008-08-05T06:39:31Z,NaN,10,What language do you use for PostgreSQL trigge...,<p>PostgreSQL is interesting in that it suppor...,sql,59345,1081.0,2008-09-12T15:39:39Z,2,<p>Skype uses <strong>PostgreSQL</strong> toge...
4,1970,116.0,2008-08-05T06:39:31Z,NaN,10,What language do you use for PostgreSQL trigge...,<p>PostgreSQL is interesting in that it suppor...,sql,69641,11116.0,2008-09-16T05:43:22Z,4,"<p>I write pretty much everything in plpgsql, ..."


In [23]:
new.shape

(216304, 13)

In [24]:
from bs4 import BeautifulSoup


In [25]:

new['Questions'] = new['Questions'].apply(lambda x: BeautifulSoup(x).get_text()) 


In [26]:
new['Answers'] = new['Answers'].apply(lambda x: BeautifulSoup(x).get_text()) 


In [27]:
new.head(2)

,Q_ID,User_id_q,Q_CreationDate,Q_ClosedDate,Q_Score,Q_Title,Questions,Tags,A_ID,User_id_a,A_CreationDate,A_Score,Answers
0,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,sql,124363,12734.0,2008-09-23T22:41:11Z,9,The Jeff Prosise version from MSDN magazine wo...
1,1970,116.0,2008-08-05T06:39:31Z,NaN,10,What language do you use for PostgreSQL trigge...,PostgreSQL is interesting in that it supports ...,sql,1971,269.0,2008-08-05T06:43:30Z,3,When I wrote my first trigger it was in Perl b...


In [28]:
import re

In [29]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [30]:
new['Questions'] = new['Questions'].apply(lambda x: clean_text(x)) 


In [31]:
new["Questions"] = new["Questions"].str.replace(r'[^\w\s]+', '_')


In [32]:
new['Answers'] = new['Answers'].apply(lambda x: clean_text(x)) 


In [33]:
new['Answers'] = new['Answers'].str.replace(r'[^\w\s]+', '_') 


In [34]:
new['Q_Title'] = new['Q_Title'].apply(lambda x: clean_text(x)) 

In [35]:
new['Q_Title'] = new['Q_Title'].str.replace(r'[^\w\s]+', '_')

In [36]:
#new['Tags'] = new['Tags'].apply(lambda x: clean_text(x))

In [37]:
#new['Tags'] = new['Tags'].str.replace(r'[^\w\s]+', '_')

In [38]:
new.head()

,Q_ID,User_id_q,Q_CreationDate,Q_ClosedDate,Q_Score,Q_Title,Questions,Tags,A_ID,User_id_a,A_CreationDate,A_Score,Answers
0,120,83.0,2008-08-01T15:50:08Z,NaN,21,asp_net site maps,has anyone got experience creating sql_based a...,sql,124363,12734.0,2008-09-23T22:41:11Z,9,the jeff prosise version from msdn magazine wo...
1,1970,116.0,2008-08-05T06:39:31Z,NaN,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,1971,269.0,2008-08-05T06:43:30Z,3,when i wrote my first trigger it was in perl b...
2,1970,116.0,2008-08-05T06:39:31Z,NaN,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,3454,277.0,2008-08-06T14:26:08Z,5,i have only really used pl_pgsql_ but that was...
3,1970,116.0,2008-08-05T06:39:31Z,NaN,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,59345,1081.0,2008-09-12T15:39:39Z,2,skype uses postgresql together with python_ an...
4,1970,116.0,2008-08-05T06:39:31Z,NaN,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,69641,11116.0,2008-09-16T05:43:22Z,4,i write pretty much everything in plpgsql_ but...


In [39]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216304 entries, 0 to 216303
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Q_ID            216304 non-null  int64  
 1   User_id_q       213269 non-null  float64
 2   Q_CreationDate  216304 non-null  object 
 3   Q_ClosedDate    10867 non-null   object 
 4   Q_Score         216304 non-null  int64  
 5   Q_Title         216304 non-null  object 
 6   Questions       216304 non-null  object 
 7   Tags            216304 non-null  object 
 8   A_ID            216304 non-null  int64  
 9   User_id_a       215061 non-null  float64
 10  A_CreationDate  216304 non-null  object 
 11  A_Score         216304 non-null  int64  
 12  Answers         216304 non-null  object 
dtypes: float64(2), int64(4), object(7)
memory usage: 23.1+ MB


In [40]:
new.rename(columns={"Q_ID":"Question_ID","User_id_q":"Id_of_user_who_asked_question","Q_Title":"Title",
                    "User_id_a":"Id_of_user_who_answered_question"},inplace = True)

In [41]:
new.drop("A_ID",axis=1,inplace = True)

In [42]:
new.drop("Question_ID",axis=1,inplace = True)

In [43]:
new.head()

,Id_of_user_who_asked_question,Q_CreationDate,Q_ClosedDate,Q_Score,Title,Questions,Tags,Id_of_user_who_answered_question,A_CreationDate,A_Score,Answers
0,83.0,2008-08-01T15:50:08Z,NaN,21,asp_net site maps,has anyone got experience creating sql_based a...,sql,12734.0,2008-09-23T22:41:11Z,9,the jeff prosise version from msdn magazine wo...
1,116.0,2008-08-05T06:39:31Z,NaN,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,269.0,2008-08-05T06:43:30Z,3,when i wrote my first trigger it was in perl b...
2,116.0,2008-08-05T06:39:31Z,NaN,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,277.0,2008-08-06T14:26:08Z,5,i have only really used pl_pgsql_ but that was...
3,116.0,2008-08-05T06:39:31Z,NaN,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,1081.0,2008-09-12T15:39:39Z,2,skype uses postgresql together with python_ an...
4,116.0,2008-08-05T06:39:31Z,NaN,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,11116.0,2008-09-16T05:43:22Z,4,i write pretty much everything in plpgsql_ but...


In [44]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216304 entries, 0 to 216303
Data columns (total 11 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Id_of_user_who_asked_question     213269 non-null  float64
 1   Q_CreationDate                    216304 non-null  object 
 2   Q_ClosedDate                      10867 non-null   object 
 3   Q_Score                           216304 non-null  int64  
 4   Title                             216304 non-null  object 
 5   Questions                         216304 non-null  object 
 6   Tags                              216304 non-null  object 
 7   Id_of_user_who_answered_question  215061 non-null  float64
 8   A_CreationDate                    216304 non-null  object 
 9   A_Score                           216304 non-null  int64  
 10  Answers                           216304 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 19.

In [45]:
new["Q_CreationDate"] = pd.to_datetime(new["Q_CreationDate"])

In [46]:
new["Q_ClosedDate"] = pd.to_datetime(new["Q_ClosedDate"])

In [47]:
new["A_CreationDate"] = pd.to_datetime(new["A_CreationDate"])

In [48]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216304 entries, 0 to 216303
Data columns (total 11 columns):
 #   Column                            Non-Null Count   Dtype              
---  ------                            --------------   -----              
 0   Id_of_user_who_asked_question     213269 non-null  float64            
 1   Q_CreationDate                    216304 non-null  datetime64[ns, UTC]
 2   Q_ClosedDate                      10867 non-null   datetime64[ns, UTC]
 3   Q_Score                           216304 non-null  int64              
 4   Title                             216304 non-null  object             
 5   Questions                         216304 non-null  object             
 6   Tags                              216304 non-null  object             
 7   Id_of_user_who_answered_question  215061 non-null  float64            
 8   A_CreationDate                    216304 non-null  datetime64[ns, UTC]
 9   A_Score                           216304 non-nul

In [49]:
#new['Q_Year'] = new['Q_CreationDate'].dt.year
#new['Q_month'] = new['Q_CreationDate'].dt.month
#new['Q_day'] = new['Q_CreationDate'].dt.day
#new['Q_time'] = new['Q_CreationDate'].dt.time

In [50]:
#new['A_Year'] = new['A_CreationDate'].dt.year
#new['A_month'] = new['A_CreationDate'].dt.month
#new['A_day'] = new['A_CreationDate'].dt.day
#new['A_time'] = new['A_CreationDate'].dt.time







In [51]:
#new.drop("A_CreationDate",axis=1,inplace=True)

In [52]:
#new.drop("Q_CreationDate",axis=1,inplace=True)

In [53]:
new.head()

,Id_of_user_who_asked_question,Q_CreationDate,Q_ClosedDate,Q_Score,Title,Questions,Tags,Id_of_user_who_answered_question,A_CreationDate,A_Score,Answers
0,83.0,2008-08-01 15:50:08+00:00,NaT,21,asp_net site maps,has anyone got experience creating sql_based a...,sql,12734.0,2008-09-23 22:41:11+00:00,9,the jeff prosise version from msdn magazine wo...
1,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,269.0,2008-08-05 06:43:30+00:00,3,when i wrote my first trigger it was in perl b...
2,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,277.0,2008-08-06 14:26:08+00:00,5,i have only really used pl_pgsql_ but that was...
3,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,1081.0,2008-09-12 15:39:39+00:00,2,skype uses postgresql together with python_ an...
4,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,11116.0,2008-09-16 05:43:22+00:00,4,i write pretty much everything in plpgsql_ but...


In [54]:
users.head()


,Unnamed: 0,id,display_name,creation_date,last_access_date,reputation,up_votes,down_votes,views
0,0,26,Shawn,2008-08-01 12:18:14.52 UTC,2018-04-11 04:40:35.66 UTC,11820,565,67,3547
1,1,26,Shawn,2008-08-01 12:18:14.52 UTC,2018-04-11 04:40:35.66 UTC,11820,565,67,3547
2,2,26,Shawn,2008-08-01 12:18:14.52 UTC,2018-04-11 04:40:35.66 UTC,11820,565,67,3547
3,3,26,Shawn,2008-08-01 12:18:14.52 UTC,2018-04-11 04:40:35.66 UTC,11820,565,67,3547
4,4,26,Shawn,2008-08-01 12:18:14.52 UTC,2018-04-11 04:40:35.66 UTC,11820,565,67,3547


In [55]:
users = users.drop("Unnamed: 0",axis=1)

In [56]:
users = users.drop_duplicates()

In [57]:
users.head()

,id,display_name,creation_date,last_access_date,reputation,up_votes,down_votes,views
0,26,Shawn,2008-08-01 12:18:14.52 UTC,2018-04-11 04:40:35.66 UTC,11820,565,67,3547
27,1109,Theo,2008-08-12 12:16:58.61 UTC,2019-08-31 11:39:52.28 UTC,116072,457,136,6905
85,364174,stats,2010-06-11 03:34:57.137 UTC,2019-08-29 18:37:35.027 UTC,325,25,1,60
87,61,Bernard,2008-08-01 14:21:00.573 UTC,2019-08-30 07:11:58.297 UTC,22779,1321,219,1731
97,20709,Black Horus,2008-09-22 20:03:39.71 UTC,2019-09-01 01:17:38.133 UTC,978,73,2,96


In [58]:
users.shape

(465460, 8)

In [59]:
new.isnull().sum()

Id_of_user_who_asked_question         3035
Q_CreationDate                           0
Q_ClosedDate                        205437
Q_Score                                  0
Title                                    0
Questions                                0
Tags                                     0
Id_of_user_who_answered_question      1243
A_CreationDate                           0
A_Score                                  0
Answers                                  0
dtype: int64

In [60]:
df = new[~new['Id_of_user_who_asked_question'].isnull()]

In [61]:
df = df[~df['Id_of_user_who_answered_question'].isnull()]

In [62]:
df.isnull().sum()

Id_of_user_who_asked_question            0
Q_CreationDate                           0
Q_ClosedDate                        201505
Q_Score                                  0
Title                                    0
Questions                                0
Tags                                     0
Id_of_user_who_answered_question         0
A_CreationDate                           0
A_Score                                  0
Answers                                  0
dtype: int64

In [63]:
df['Id_of_user_who_answered_question'] = df["Id_of_user_who_answered_question"].astype(int)      

In [64]:
df.head()

,Id_of_user_who_asked_question,Q_CreationDate,Q_ClosedDate,Q_Score,Title,Questions,Tags,Id_of_user_who_answered_question,A_CreationDate,A_Score,Answers
0,83.0,2008-08-01 15:50:08+00:00,NaT,21,asp_net site maps,has anyone got experience creating sql_based a...,sql,12734,2008-09-23 22:41:11+00:00,9,the jeff prosise version from msdn magazine wo...
1,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,269,2008-08-05 06:43:30+00:00,3,when i wrote my first trigger it was in perl b...
2,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,277,2008-08-06 14:26:08+00:00,5,i have only really used pl_pgsql_ but that was...
3,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,1081,2008-09-12 15:39:39+00:00,2,skype uses postgresql together with python_ an...
4,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,11116,2008-09-16 05:43:22+00:00,4,i write pretty much everything in plpgsql_ but...


In [65]:
df.rename(columns={"Id_of_user_who_answered_question":"id"},inplace = True)

In [66]:
df.head()

,Id_of_user_who_asked_question,Q_CreationDate,Q_ClosedDate,Q_Score,Title,Questions,Tags,id,A_CreationDate,A_Score,Answers
0,83.0,2008-08-01 15:50:08+00:00,NaT,21,asp_net site maps,has anyone got experience creating sql_based a...,sql,12734,2008-09-23 22:41:11+00:00,9,the jeff prosise version from msdn magazine wo...
1,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,269,2008-08-05 06:43:30+00:00,3,when i wrote my first trigger it was in perl b...
2,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,277,2008-08-06 14:26:08+00:00,5,i have only really used pl_pgsql_ but that was...
3,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,1081,2008-09-12 15:39:39+00:00,2,skype uses postgresql together with python_ an...
4,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,11116,2008-09-16 05:43:22+00:00,4,i write pretty much everything in plpgsql_ but...


In [67]:
Dataset = df.merge(users, on='id')

In [68]:
Dataset.head(3)

,Id_of_user_who_asked_question,Q_CreationDate,Q_ClosedDate,Q_Score,Title,Questions,Tags,id,A_CreationDate,A_Score,Answers,display_name,creation_date,last_access_date,reputation,up_votes,down_votes,views
0,83.0,2008-08-01 15:50:08+00:00,NaT,21,asp_net site maps,has anyone got experience creating sql_based a...,sql,12734,2008-09-23 22:41:11+00:00,9,the jeff prosise version from msdn magazine wo...,Kelly Adams,2008-09-16 15:27:55.78 UTC,2019-08-29 19:59:10.743 UTC,695,35,0,105
1,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,269,2008-08-05 06:43:30+00:00,3,when i wrote my first trigger it was in perl b...,sparkes,2008-08-04 10:13:44.767 UTC,2008-09-30 07:30:19.52 UTC,14710,330,37,549
2,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,277,2008-08-06 14:26:08+00:00,5,i have only really used pl_pgsql_ but that was...,kaybenleroll,2008-08-04 10:55:22.093 UTC,2019-08-29 13:42:38.157 UTC,7980,878,5,534


In [69]:
Dataset = Dataset.drop(['creation_date','last_access_date','reputation','up_votes','down_votes','views'],axis=1)

In [70]:
Dataset.head(2)

,Id_of_user_who_asked_question,Q_CreationDate,Q_ClosedDate,Q_Score,Title,Questions,Tags,id,A_CreationDate,A_Score,Answers,display_name
0,83.0,2008-08-01 15:50:08+00:00,NaT,21,asp_net site maps,has anyone got experience creating sql_based a...,sql,12734,2008-09-23 22:41:11+00:00,9,the jeff prosise version from msdn magazine wo...,Kelly Adams
1,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,269,2008-08-05 06:43:30+00:00,3,when i wrote my first trigger it was in perl b...,sparkes


In [71]:
Dataset.rename(columns={"display_name":"Answers given by"},inplace = True)

In [72]:
users.rename(columns={'id':"Id_of_user_who_asked_question"},inplace=True)

In [73]:
Dataset = Dataset.merge(users,on="Id_of_user_who_asked_question")

In [74]:
Dataset.head(2)

,Id_of_user_who_asked_question,Q_CreationDate,Q_ClosedDate,Q_Score,Title,Questions,Tags,id,A_CreationDate,A_Score,Answers,Answers given by,display_name,creation_date,last_access_date,reputation,up_votes,down_votes,views
0,83.0,2008-08-01 15:50:08+00:00,NaT,21,asp_net site maps,has anyone got experience creating sql_based a...,sql,12734,2008-09-23 22:41:11+00:00,9,the jeff prosise version from msdn magazine wo...,Kelly Adams,Zack Peterson,2008-08-01 16:31:56.94 UTC,2019-08-27 20:25:33.09 UTC,31853,1966,63,2886
1,116.0,2008-08-05 06:39:31+00:00,NaT,10,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,269,2008-08-05 06:43:30+00:00,3,when i wrote my first trigger it was in perl b...,sparkes,Mark Harrison,2008-08-02 05:51:57.153 UTC,2019-09-01 02:39:46.78 UTC,181436,2537,39,19386


In [75]:
Dataset.rename(columns={"display_name":"Questions asked by"},inplace = True)

In [76]:
Dataset = Dataset.drop(['creation_date','last_access_date','reputation','up_votes','down_votes','views'],axis=1)

In [77]:
Dataset = Dataset.drop(['Id_of_user_who_asked_question','Q_Score',"A_Score"],axis=1)

In [78]:
Dataset.head(3)

,Q_CreationDate,Q_ClosedDate,Title,Questions,Tags,id,A_CreationDate,Answers,Answers given by,Questions asked by
0,2008-08-01 15:50:08+00:00,NaT,asp_net site maps,has anyone got experience creating sql_based a...,sql,12734,2008-09-23 22:41:11+00:00,the jeff prosise version from msdn magazine wo...,Kelly Adams,Zack Peterson
1,2008-08-05 06:39:31+00:00,NaT,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,269,2008-08-05 06:43:30+00:00,when i wrote my first trigger it was in perl b...,sparkes,Mark Harrison
2,2008-08-05 06:39:31+00:00,NaT,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,277,2008-08-06 14:26:08+00:00,i have only really used pl_pgsql_ but that was...,kaybenleroll,Mark Harrison


In [79]:
Dataset = Dataset.drop("Q_ClosedDate",axis=1)

In [80]:
Dataset.head(2)

,Q_CreationDate,Title,Questions,Tags,id,A_CreationDate,Answers,Answers given by,Questions asked by
0,2008-08-01 15:50:08+00:00,asp_net site maps,has anyone got experience creating sql_based a...,sql,12734,2008-09-23 22:41:11+00:00,the jeff prosise version from msdn magazine wo...,Kelly Adams,Zack Peterson
1,2008-08-05 06:39:31+00:00,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,269,2008-08-05 06:43:30+00:00,when i wrote my first trigger it was in perl b...,sparkes,Mark Harrison


In [81]:
Dataset["Estimate_Time"] = Dataset["A_CreationDate"]-Dataset["Q_CreationDate"]

In [82]:
Dataset.head()

,Q_CreationDate,Title,Questions,Tags,id,A_CreationDate,Answers,Answers given by,Questions asked by,Estimate_Time
0,2008-08-01 15:50:08+00:00,asp_net site maps,has anyone got experience creating sql_based a...,sql,12734,2008-09-23 22:41:11+00:00,the jeff prosise version from msdn magazine wo...,Kelly Adams,Zack Peterson,53 days 06:51:03
1,2008-08-05 06:39:31+00:00,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,269,2008-08-05 06:43:30+00:00,when i wrote my first trigger it was in perl b...,sparkes,Mark Harrison,0 days 00:03:59
2,2008-08-05 06:39:31+00:00,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,277,2008-08-06 14:26:08+00:00,i have only really used pl_pgsql_ but that was...,kaybenleroll,Mark Harrison,1 days 07:46:37
3,2008-08-05 06:39:31+00:00,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,1081,2008-09-12 15:39:39+00:00,skype uses postgresql together with python_ an...,dpavlin,Mark Harrison,38 days 09:00:08
4,2008-08-05 06:39:31+00:00,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,11116,2008-09-16 05:43:22+00:00,i write pretty much everything in plpgsql_ but...,decibel,Mark Harrison,41 days 23:03:51


In [83]:
Dataset.drop(["Q_CreationDate","A_CreationDate","id"],axis=1, inplace=True)

In [84]:
Dataset.head()

,Title,Questions,Tags,Answers,Answers given by,Questions asked by,Estimate_Time
0,asp_net site maps,has anyone got experience creating sql_based a...,sql,the jeff prosise version from msdn magazine wo...,Kelly Adams,Zack Peterson,53 days 06:51:03
1,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,when i wrote my first trigger it was in perl b...,sparkes,Mark Harrison,0 days 00:03:59
2,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,i have only really used pl_pgsql_ but that was...,kaybenleroll,Mark Harrison,1 days 07:46:37
3,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,skype uses postgresql together with python_ an...,dpavlin,Mark Harrison,38 days 09:00:08
4,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,i write pretty much everything in plpgsql_ but...,decibel,Mark Harrison,41 days 23:03:51


In [85]:
Dataset.isnull().sum()

Title                  0
Questions              0
Tags                   0
Answers                0
Answers given by       2
Questions asked by    12
Estimate_Time          0
dtype: int64

In [86]:
Dataset = Dataset[~Dataset["Answers given by"].isnull()]

In [87]:
Dataset = Dataset[~Dataset["Questions asked by"].isnull()]

In [89]:
Dataset.isnull().sum()

Title                 0
Questions             0
Tags                  0
Answers               0
Answers given by      0
Questions asked by    0
Estimate_Time         0
dtype: int64

When a user raises a question with a data science tag ('python', 'r', 'matlab', 'sas', 'excel', 'sql'), then Stack Overflow sends the notification to the relevant users.



In [95]:
Dataset.head()

,Title,Questions,Tags,Answers,Answers given by,Questions asked by,Estimate_Time
0,asp_net site maps,has anyone got experience creating sql_based a...,sql,the jeff prosise version from msdn magazine wo...,Kelly Adams,Zack Peterson,53 days 06:51:03
1,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,when i wrote my first trigger it was in perl b...,sparkes,Mark Harrison,0 days 00:03:59
2,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,i have only really used pl_pgsql_ but that was...,kaybenleroll,Mark Harrison,1 days 07:46:37
3,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,skype uses postgresql together with python_ an...,dpavlin,Mark Harrison,38 days 09:00:08
4,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,i write pretty much everything in plpgsql_ but...,decibel,Mark Harrison,41 days 23:03:51


In [126]:
Obj1 = pd.pivot_table(Dataset, values =['Answers given by'],index=["Tags"],aggfunc=lambda x: ', '.join(x))


In [127]:
Obj1

,Answers given by
Tags,
excel,"gserg, jeffo, adarsha, michael mrozek, msmucke..."
excel matlab,"amro, user2991243, daniel pereira, amro, buzjw..."
excel sas,"allan bowe, zach, chang chung, martin bã_â_gel..."
matlab,"kostas, gbn, jonas, jonas, jonas, mark elliot,..."
python,"ignacio vazquez_abrams, alex martelli, john la..."
python excel,"s_lott, kylotan, rossfabricant, jeff youel, un..."
python excel r,hansi
python matlab,"dzhelil, robert karl, tom10, peter, pv_, smart..."
python r,"john, joshua ulrich, vincent zoonekynd, parfai..."


In [130]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95041 entries, 0 to 95053
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   Title               95041 non-null  object         
 1   Questions           95041 non-null  object         
 2   Tags                95041 non-null  object         
 3   Answers             95041 non-null  object         
 4   Answers given by    95041 non-null  object         
 5   Questions asked by  95041 non-null  object         
 6   Estimate_Time       95041 non-null  timedelta64[ns]
dtypes: object(6), timedelta64[ns](1)
memory usage: 8.3+ MB


In [149]:
Dataset['Time_in_sec'] = Dataset['Estimate_Time'].dt.total_seconds()


In [155]:
Dataset.head()

,Title,Questions,Tags,Answers,Answers given by,Questions asked by,Estimate_Time,Time_in_sec
0,asp_net site maps,has anyone got experience creating sql_based a...,sql,the jeff prosise version from msdn magazine wo...,kelly adams,Zack Peterson,53 days 06:51:03,4603863.0
1,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,when i wrote my first trigger it was in perl b...,sparkes,Mark Harrison,0 days 00:03:59,239.0
2,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,i have only really used pl_pgsql_ but that was...,kaybenleroll,Mark Harrison,1 days 07:46:37,114397.0
3,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,skype uses postgresql together with python_ an...,dpavlin,Mark Harrison,38 days 09:00:08,3315608.0
4,what language do you use for postgresql trigge...,postgresql is interesting in that it supports ...,sql,i write pretty much everything in plpgsql_ but...,decibel,Mark Harrison,41 days 23:03:51,3625431.0


In [157]:
Obj2 = pd.pivot_table(Dataset, values =['Time_in_sec'],index=["Tags"],aggfunc=np.median, fill_value=pd.Timedelta(hours=0),
                      margins=True,margins_name="TOTAL")


In [158]:
Obj2.

,Time_in_sec
Tags,
excel,5423.0
excel matlab,12257.0
excel sas,1183884.5
matlab,4462.0
python,2530.0
python excel,6905.0
python excel r,7832.0
python matlab,3973.5
python r,8594.0


In [163]:
x = Obj1.merge(Obj2,left_index=True,right_index=True)

In [164]:
x


,Answers given by,Time_in_sec
Tags,,
excel,"gserg, jeffo, adarsha, michael mrozek, msmucke...",5423.0
excel matlab,"amro, user2991243, daniel pereira, amro, buzjw...",12257.0
excel sas,"allan bowe, zach, chang chung, martin bã_â_gel...",1183884.5
matlab,"kostas, gbn, jonas, jonas, jonas, mark elliot,...",4462.0
python,"ignacio vazquez_abrams, alex martelli, john la...",2530.0
python excel,"s_lott, kylotan, rossfabricant, jeff youel, un...",6905.0
python excel r,hansi,7832.0
python matlab,"dzhelil, robert karl, tom10, peter, pv_, smart...",3973.5
python r,"john, joshua ulrich, vincent zoonekynd, parfai...",8594.0


In [166]:
x.to_excel("filename.xlsx")
